# Задача 9 
___
Реализуйте вычисление контрольной суммы для каждой буквы сообщения:
* На вход передаётся некоторое сообщение.
* Вы разбиваете его на символы, вычисляете crc8 для каждого символа, выводите символ и его контрольную сумму в виде 0 и 1, то есть в бинарной форме.
* Создаёте декодер для получения сообщения, проверки контрольной суммы для каждого символа.
* В случае обнаружения проблем с контрольной суммой выводите на консоль и в файл ошибку. Также передаёте тело сообщения и контрольную сумму.

### Входные данные
```shell
Введите тип (1 — кодер и декодер, 2 — декодер): 1
Введите сообщение: hello
```
### Вывод программы
```shell
Кодированное сообщение: 01101000 00011111 01100101 00111100 01101100 00000011 01101100 00000011 01101111 00001010
Декодированное сообщение: hello
```

### Входные данные
```shell
Введите тип (1 — кодер и декодер, 2 — декодер): 2
Введите кодированное сообщение: 01101000 00011111 01100101 00111100 01101100 00000011 01101100 00000011 01101111 00001010
```
### Вывод программы
```shell
Все контрольные суммы верны.
Декодированное сообщение: hello
```

### Входные данные
```shell
Введите тип (1 — кодер и декодер, 2 — декодер): 2
```
### Вывод программы
```shell
Введите кодированное сообщение: 01101000 00111111 01100101 00111100 01101100 00000011 01101100 00000011 01101111 00001010
Ошибка в символе h с индексом 1.
Символ: h
crc8: 1f
crc8 (полученное): 3f
Декодированное сообщение: hello
```
___
# Решение

In [2]:
from loguru import logger

# Настройка логирования
logger.add("crc_errors.log", format="{time} | {level} | {message}", level="ERROR")

# Таблица CRC8
CRC8_TABLE = [
    0x00, 0x07, 0x0E, 0x09, 0x1C, 0x1B, 0x12, 0x15,
    0x38, 0x3F, 0x36, 0x31, 0x24, 0x23, 0x2A, 0x2D,
    0x70, 0x77, 0x7E, 0x79, 0x6C, 0x6B, 0x62, 0x65,
    0x48, 0x4F, 0x46, 0x41, 0x54, 0x53, 0x5A, 0x5D,
    0xE0, 0xE7, 0xEE, 0xE9, 0xFC, 0xFB, 0xF2, 0xF5,
    0xD8, 0xDF, 0xD6, 0xD1, 0xC4, 0xC3, 0xCA, 0xCD,
    0x90, 0x97, 0x9E, 0x99, 0x8C, 0x8B, 0x82, 0x85,
    0xA8, 0xAF, 0xA6, 0xA1, 0xB4, 0xB3, 0xBA, 0xBD,
    0xC7, 0xC0, 0xC9, 0xCE, 0xDB, 0xDC, 0xD5, 0xD2,
    0xFF, 0xF8, 0xF1, 0xF6, 0xE3, 0xE4, 0xED, 0xEA,
    0xB7, 0xB0, 0xB9, 0xBE, 0xAB, 0xAC, 0xA5, 0xA2,
    0x8F, 0x88, 0x81, 0x86, 0x93, 0x94, 0x9D, 0x9A,
    0x27, 0x20, 0x29, 0x2E, 0x3B, 0x3C, 0x35, 0x32,
    0x1F, 0x18, 0x11, 0x16, 0x03, 0x04, 0x0D, 0x0A,
    0x57, 0x50, 0x59, 0x5E, 0x4B, 0x4C, 0x45, 0x42,
    0x6F, 0x68, 0x61, 0x66, 0x73, 0x74, 0x7D, 0x7A,
    0x89, 0x8E, 0x87, 0x80, 0x95, 0x92, 0x9B, 0x9C,
    0xB1, 0xB6, 0xBF, 0xB8, 0xAD, 0xAA, 0xA3, 0xA4,
    0xF9, 0xFE, 0xF7, 0xF0, 0xE5, 0xE2, 0xEB, 0xEC,
    0xC1, 0xC6, 0xCF, 0xC8, 0xDD, 0xDA, 0xD3, 0xD4,
    0x69, 0x6E, 0x67, 0x60, 0x75, 0x72, 0x7B, 0x7C,
    0x51, 0x56, 0x5F, 0x58, 0x4D, 0x4A, 0x43, 0x44,
    0x19, 0x1E, 0x17, 0x10, 0x05, 0x02, 0x0B, 0x0C,
    0x21, 0x26, 0x2F, 0x28, 0x3D, 0x3A, 0x33, 0x34,
    0x4E, 0x49, 0x40, 0x47, 0x52, 0x55, 0x5C, 0x5B,
    0x76, 0x71, 0x78, 0x7F, 0x6A, 0x6D, 0x64, 0x63,
    0x3E, 0x39, 0x30, 0x37, 0x22, 0x25, 0x2C, 0x2B,
    0x06, 0x01, 0x08, 0x0F, 0x1A, 0x1D, 0x14, 0x13,
    0xAE, 0xA9, 0xA0, 0xA7, 0xB2, 0xB5, 0xBC, 0xBB,
    0x96, 0x91, 0x98, 0x9F, 0x8A, 0x8D, 0x84, 0x83,
    0xDE, 0xD9, 0xD0, 0xD7, 0xC2, 0xC5, 0xCC, 0xCB,
    0xE6, 0xE1, 0xE8, 0xEF, 0xFA, 0xFD, 0xF4, 0xF3
]

def crc8(data):
    crc = 0
    for byte in data:
        crc = CRC8_TABLE[crc ^ byte]
    return crc

def encode_message(message):
    encoded_message = []
    for char in message:
        char_code = ord(char)
        crc = crc8([char_code])
        encoded_message.append(f"{char_code:08b} {crc:08b}")
    return ' '.join(encoded_message)

def decode_message(encoded_message):
    parts = encoded_message.split()
    decoded_message = []
    errors = []
    for i in range(0, len(parts), 2):
        char_code = int(parts[i], 2)
        received_crc = int(parts[i + 1], 2)
        calculated_crc = crc8([char_code])
        if received_crc != calculated_crc:
            errors.append((i // 2, chr(char_code), received_crc, calculated_crc))
        decoded_message.append(chr(char_code))
    return ''.join(decoded_message), errors

def main():
    mode = int(input("Введите тип (1 — кодер и декодер, 2 — декодер): "))
    
    if mode == 1:
        message = input("Введите сообщение: ")
        encoded_message = encode_message(message)
        print(f"Кодированное сообщение: {encoded_message}")
        
        decoded_message, errors = decode_message(encoded_message)
        if errors:
            for index, char, received_crc, calculated_crc in errors:
                logger.error(f"Ошибка в символе {char} с индексом {index}. Символ: {char} crc8: {received_crc:02x} crc8 (полученное): {calculated_crc:02x}")
            print("Обнаружены ошибки в контрольной сумме.")
        else:
            print("Все контрольные суммы верны.")
        print(f"Декодированное сообщение: {decoded_message}")
    
    elif mode == 2:
        encoded_message = input("Введите кодированное сообщение: ")
        decoded_message, errors = decode_message(encoded_message)
        if errors:
            for index, char, received_crc, calculated_crc in errors:
                logger.error(f"Ошибка в символе {char} с индексом {index}. Символ: {char} crc8: {received_crc:02x} crc8 (полученное): {calculated_crc:02x}")
            print("Обнаружены ошибки в контрольной сумме.")
        else:
            print("Все контрольные суммы верны.")
        print(f"Декодированное сообщение: {decoded_message}")

if __name__ == "__main__":
    main()

Кодированное сообщение: 01101000 00011111 01100101 00111100 01101100 00000011 01101100 00000011 01101111 00001010
Все контрольные суммы верны.
Декодированное сообщение: hello


#### Этот код выполняет следующие действия:
* Настраивает логирование с помощью `loguru`.
* Определяет функцию `crc8` для вычисления контрольной суммы `CRC8`.
* Определяет функцию `encode_message` для кодирования сообщения, добавляя контрольную сумму `CRC8` к каждому символу.
* Определяет функцию `decode_message` для декодирования сообщения и проверки контрольных сумм.
* В основной функции main вводится тип операции (кодер и декодер или только декодер), затем вводится сообщение или закодированное сообщение. В зависимости от выбранного режима, сообщение кодируется и декодируется, или только декодируется. Обнаруженные ошибки выводятся на консоль и записываются в лог-файл.
